In [3]:
import math

In [4]:
def keys_dist(letter1, letter2):
    distance = 19.05
    letters = "qazwsxedcrfvtgbyhnujmik,ol.p;/"
    bias = {0: 0, 1: 0.25 * distance, 2: 0.75 * distance}
    
    letter1_id = letters.find(letter1.lower())
    letter2_id = letters.find(letter2.lower())
    
    coords1 = (bias[letter1_id % 3] + (letter1_id // 3) * distance, (letter1_id % 3) * distance)
    coords2 = (bias[letter2_id % 3] + (letter2_id // 3) * distance, (letter2_id % 3) * distance)
    
    evc_dist = ((coords1[0] - coords2[0]) ** 2 + (coords1[1] - coords2[1]) ** 2) ** 0.5
    
    return evc_dist

In [5]:
test_cases = [["A", "B", 87.82],
              ["B", "Q", 98.18],
              ["G", "C", 34.34],
              ["H", "J", 19.05],
              ["L", "J", 38.10],
              ["P", "X", 143.27],
              ["Y", "Y", 0]]

for case in test_cases:
    if abs(keys_dist(case[0], case[1]) - case[2]) < 0.01:
        print("passed")
    else:
        print(keys_dist(case[0], case[1]), case)

passed
passed
passed
passed
passed
passed
passed


In [6]:
def wagner_fisher(str1, str2, del_ins_cost):
    m = len(str1)
    n = len(str2)

    d = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    for i in range(m + 1):
        d[i][0] = i * del_ins_cost
    for j in range(n + 1):
        d[0][j] = j * del_ins_cost

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                subcost = 0
            else:
                subcost = keys_dist(str1[i - 1], str2[j - 1]) / 10
                
            d[i][j] = min(d[i - 1][j - 1] + subcost,
                          d[i - 1][j] + del_ins_cost,
                          d[i][j - 1] + del_ins_cost)

    return d[m][n]


str1 = "put"
str2 = "pet"
print("Edit distance between '{}' and '{}': {}".format(str1, str2, wagner_fisher(str1, str2, 2)))

Edit distance between 'put' and 'pet': 4


In [7]:
def lev(a, b, del_cost, ins_cost, sub_cost, trans_cost):
    if len(a) == 0 or len(b) == 0:
        return max(len(a), len(b)) * del_cost
    if a[0] == b[0]:
        return lev(a[1:], b[1:], del_cost, ins_cost, sub_cost, trans_cost)
    else:
        return min(lev(a[1:], b, del_cost, ins_cost, sub_cost, trans_cost) + del_cost,
                   lev(a, b[1:], del_cost, ins_cost, sub_cost, trans_cost) + ins_cost,
                   lev(a[1:], b[1:], del_cost, ins_cost, sub_cost, trans_cost) + sub_cost,
                   lev(a[2:], b[2:], del_cost, ins_cost, sub_cost, trans_cost) + trans_cost if (len(a) > 1 and len(b) > 1) and (a[0] == b[1] and a[1] == b[0]) else math.inf)


print(lev("pet", "pte", 1, 1, 2, 0))

0


In [8]:
def wagner_fisher_with_trans(str1, str2, del_ins_cost, trans_cost):
    m = len(str1)
    n = len(str2)

    d = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    for i in range(m + 1):
        d[i][0] = i * del_ins_cost
    for j in range(n + 1):
        d[0][j] = j * del_ins_cost

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                subcost = 0
            else:
                subcost = keys_dist(str1[i - 1], str2[j - 1]) / 10
            
            if (i > 1 and j > 1) and (str1[i - 1] == str2[i - 2] and str1[i - 2] == str2[i - 1]):
                trans_cost_tmp = trans_cost
            else:
                trans_cost_tmp = math.inf
                
            d[i][j] = min(d[i - 1][j - 1] + subcost,
                          d[i - 1][j] + del_ins_cost,
                          d[i][j - 1] + del_ins_cost,
                          d[i - 2][j - 2] + trans_cost_tmp)

    return d[m][n]


str1 = "put"
str2 = "ptu"
print("Edit distance between '{}' and '{}': {}".format(str1, str2, wagner_fisher_with_trans(str1, str2, 2, 0)))

Edit distance between 'put' and 'ptu': 0


In [9]:
import nltk
from nltk import book
import numpy as np

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [10]:
def get_vocab(corpus):
    vocab = {}
    for i, word in enumerate(set(corpus), 1):
        vocab[word] = i
    return vocab

In [46]:
t1 = book.text6
t2 = book.text1

In [47]:
vocab = get_vocab(t1.tokens + t2.tokens)

In [48]:
len(vocab)

20031

In [49]:
def ohe(sentence, vocab):
    ohe_sentence = np.array([0 for _ in range(len(vocab) + 1)])
    for word in sentence:
        ohe_sentence[vocab.get(word, 0)] += 1
        
    return ohe_sentence

In [50]:
ohe_t1 = ohe(t1.tokens, vocab)
ohe_t2 = ohe(t2.tokens, vocab)

In [51]:
(ohe_t1 @ ohe_t2) / np.linalg.norm(ohe_t1) / np.linalg.norm(ohe_t2)

0.5477550016740712